> ### Import Library

In [ ]:
pip install pandas-gbq google-cloud-bigquery

In [ ]:
import pandas as pd
import numpy as np
from openpyxl.styles import Font

data = pd.read_excel('jobs-report 22 Desember 2025.xlsx')

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


> ### Scoring

In [ ]:
import pandas as pd
import numpy as np
from openpyxl.styles import Font

# =========================
# 1) Urutan Worker
# =========================
worker_order = [
    "Ema Pangestika","Alya Rizqi Salsabila","Vianty Ayu Lestary","Novanda Nurliani Putri","Dona Palupi Utari",
    "Anggi Ardiana Saputri","Eti Suharni","Amalia Cahya Anggraheni","Yufiatun Nurkhasanah","Amira Meliana Putri",
    "Tri Baroroh Ratnaningsih","Sri Wigati","Laela Dwi Puspita Sari","Elfa Sari Pambayun","Shinta Maharani Putri",
    "Endah Rahma Setyowati","Ainun Musyarofah","Eva Aprilia Dewi","Tri Setio Utami","Ravinda Tyas Kusumaningrum",
    "Ratna Aji Prastika","Rini Wijayanti","Fina Udhul Masitoh","Anggita Eka Putri","Galuh Lalitadhira Anindita",
    "Intan Supraptiningsih","Rizky Nuramalia","Etri Nur Susanti","Dwi Pangesti","Hani Herlina Agne Saputri",
    "Ummu Mardiyatun","Fitria Prila Rinawati","Lisa Kurniati","Adinda Nur Indriani","Nawang Sasi Chotniati",
    "Yuyun Khikmatin","Elia Vionita","Al Asri Dwi Salasih","Yutika Praningsih","Melina Nur Afifah",
    "Raflina Nurwanda Ningsih","Uti Andriyani","Dea Dwi Oktianti","Awis Damayanti","Safira Almadani",
    "Nabila Eka Rahayu","Ani Kurniawati","Rena Wulandari","Alfi Ayuningtias","Kharisma Salsabila",
    "Dicki Mardiyanti","Ashilla Nadiya Amany","Sofwa Aulia Rahmawati","Sarah Faiqoh Nugroho",
    "Intan Nur Laelasari","Chintya Feronica"
]

# =========================
# 2) Normalisasi Nama Worker
# =========================
def normalisasi_worker(df):
    df['Worker'] = (
        df['Worker']
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace(r'\s+', ' ', regex=True)
    )
    return df

# =========================
# 3) Performance Score
# =========================
def hitung_score(data):
    df = normalisasi_worker(data.copy())
    df['Status'] = df['Status'].astype(str).str.strip().str.title()

    status_counts = df.groupby(['Worker','Status']).size().unstack(fill_value=0).reset_index()

    for col in ['Success','Scheduled','Issue']:
        if col not in status_counts.columns:
            status_counts[col] = 0

    status_counts['total_jobs'] = status_counts['Success'] + status_counts['Issue'] + status_counts['Scheduled']

    summary = status_counts[['Worker','Success','Issue','Scheduled','total_jobs']].copy()
    summary['Score'] = summary.apply(
        lambda x: x['Success'] / (x['total_jobs'] - x['Issue']) if (x['total_jobs'] - x['Issue']) > 0 else np.nan,
        axis=1
    )

    summary['Final Score'] = (summary['Score'] * 100).round(2)
    summary['Worker'] = summary['Worker'].str.title()

    all_workers = pd.DataFrame({'Worker': [w.title() for w in worker_order]})
    summary = all_workers.merge(summary, on='Worker', how='left').fillna(0)

    print(f"✅ Performance Score dihitung untuk {summary['Worker'].nunique()} worker")
    return summary[['Worker','Success','Scheduled','Issue','total_jobs','Score','Final Score']]

# =========================
# 4) Completeness Score (Check-in / Check-out)
# =========================
def hitung_completeness(data):
    df = normalisasi_worker(data.copy())
    df['Status'] = df['Status'].astype(str).str.strip().str.title()
    df = df[df['Status'] == 'Success'].copy()

    for col in ['Check-in', 'Check-out']:
        if col not in df.columns:
            raise ValueError(f"Kolom {col} tidak ditemukan!")

    summary = df.groupby('Worker').agg(
        total_jobs=('Worker', 'count'),
        checkin_jobs=('Check-in', lambda x: x.notna().sum()),
        checkout_jobs=('Check-out', lambda x: x.notna().sum())
    ).reset_index()

    summary['persen_checkin'] = (summary['checkin_jobs'] / summary['total_jobs'] * 100)
    summary['persen_checkout'] = (summary['checkout_jobs'] / summary['total_jobs'] * 100)
    summary['score'] = ((summary['persen_checkin'] + summary['persen_checkout']) / 2).round(2)
    summary['Worker'] = summary['Worker'].str.title()

    all_workers = pd.DataFrame({'Worker': [w.title() for w in worker_order]})
    summary = all_workers.merge(summary, on='Worker', how='left').fillna(0)

    print(f"✅ Completeness dihitung untuk {summary['Worker'].nunique()} worker")
    return summary[['Worker','total_jobs','checkin_jobs','checkout_jobs','persen_checkin','persen_checkout','score']]

# =========================
# 5) DWS = Rata-rata dari Performance & Completeness
# =========================
def hitung_dws(perf_df, comp_df):
    perf_df = normalisasi_worker(perf_df.copy())
    comp_df = normalisasi_worker(comp_df.copy())

    merged = (
        perf_df[['Worker','Final Score']]
        .merge(comp_df[['Worker','score']], on='Worker', how='outer')
    )

    merged['Final Score'] = merged['Final Score'].astype(float)
    merged['score'] = merged['score'].astype(float)
    merged['DWS'] = ((merged['Final Score'] + merged['score']) / 2).round(2)

    merged['Worker'] = merged['Worker'].str.title()
    merged = merged.sort_values('DWS', ascending=False).reset_index(drop=True)
    merged['Ranking'] = np.arange(1, len(merged)+1)

    print(f"✅ DWS (rata-rata Performance & Completeness) dihitung untuk {merged['Worker'].nunique()} worker")
    return merged[['Worker','Final Score','score','DWS','Ranking']]

In [ ]:
# Ambil semua tanggal unik dari kolom 'Date' di data
date_list = data['Date'].dropna().unique().tolist()

# Filter data berdasarkan semua tanggal tersebut
filtered_data = data[data['Date'].isin(date_list)].copy()

# Hitung status per Worker, Date
status_counts = (
    filtered_data.groupby(['Worker', 'Date', 'Status'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Pastikan kolom 'Success', 'Scheduled', 'Issue' ada (isi 0 jika tidak ada)
for col in ['Success', 'Scheduled', 'Issue']:
    if col not in status_counts.columns:
        status_counts[col] = 0

# ✅ Total jobs = Success + Issue
status_counts['total_jobs'] = status_counts['Success'] + status_counts['Issue']

# Gabungkan per Worker (agregasi total)
worker_summary = (
    status_counts.groupby('Worker', as_index=False)[['Success', 'Issue', 'total_jobs']]
    .sum()
)

# ✅ Hitung Score persis seperti di Excel: Success / (total_jobs - Issue)
# Hindari pembagian dengan nol: kalau hasilnya nol, ganti dengan NaN atau 0
worker_summary['Score'] = worker_summary.apply(
    lambda x: x['Success'] / (x['total_jobs'] - x['Issue'])
    if (x['total_jobs'] - x['Issue']) > 0 else 0,
    axis=1
).round(2)

# ✅ Final Score = Score * 100
worker_summary['Final Score'] = (worker_summary['Score'] * 100).round(2)

# Tampilkan hasil akhir
print(worker_summary[['Worker', 'Success', 'Issue', 'total_jobs', 'Score', 'Final Score']].head())


Status                Worker  Success  Issue  total_jobs  Score  Final Score
0        AMIRA MELIANA PUTRI        3      1           4    1.0        100.0
1           Ainun Musyarofah        3      1           4    1.0        100.0
2        Al Asri Dwi Salasih        2      0           2    1.0        100.0
3           Alfi Ayuningtias        4      0           4    1.0        100.0
4       Alya Rizqi Salsabila        0      0           0    0.0          0.0


> ### Check In / Out

In [ ]:
import re
def extract_kurung_datetime(s, tanggal=None, tahun_default='2025'):
    if pd.isnull(s): return pd.NaT
    s = str(s).strip()
    match_kurung = re.match(r'\((\d{2}) (\w{3}) (\d{2}):(\d{2})\)', s)
    if match_kurung:
        day, month, hour, minute = match_kurung.groups()
        tahun = tahun_default
        return pd.to_datetime(f"{tahun}-{month}-{day} {hour}:{minute}", format="%Y-%b-%d %H:%M", errors='coerce')
    match_comb = re.search(r'\((\d{2}) (\w{3}) (\d{2}):(\d{2})\)', s)
    if match_comb:
        day, month, hour, minute = match_comb.groups()
        tahun = tahun_default
        return pd.to_datetime(f"{tahun}-{month}-{day} {hour}:{minute}", format="%Y-%b-%d %H:%M", errors='coerce')
    # Jam tunggal
    match_jam = re.match(r'^(\d{2}):(\d{2})$', s)
    if match_jam and tanggal is not None:
        if isinstance(tanggal, pd.Timestamp):
            day = tanggal.day
            month = tanggal.strftime('%b')
            tahun = tanggal.year
        else:
            parts = str(tanggal).split()
            if len(parts) == 3:
                day, month, tahun = parts
            elif len(parts) == 2:
                day, month = parts
                tahun = tahun_default
            else:
                year = tahun_default
        hour, minute = match_jam.groups()
        return pd.to_datetime(f"{tahun}-{month}-{day} {hour}:{minute}", format="%Y-%b-%d %H:%M", errors='coerce')
    return pd.NaT

def format_jam_menit(menit_total):
    if pd.isnull(menit_total) or menit_total <= 0:
        return "0:00 jam"
    jam = int(menit_total // 60)
    menit = int(menit_total % 60)
    return f"{jam}:{menit:02d} jam"

def format_menit(menit_float):
    if pd.isnull(menit_float) or menit_float <= 0:
        return "0:00 menit"
    total_seconds = int(round(menit_float * 60))
    menit = total_seconds // 60
    detik = total_seconds % 60
    return f"{menit}:{detik:02d} menit"

if not isinstance(data, pd.DataFrame):
    try:
        data = pd.DataFrame(data)  # Coba konversi jika list atau array dah
    except:
        raise ValueError("Gagal mengonversi 'data' ke DataFrame. Pastikan 'data' adalah list of dicts atau struktur yang valid.")

if 'Date' not in data.columns:
    raise KeyError("Kolom 'Date' tidak ditemukan di data. Periksa DataFrame Anda.")
if data['Date'].dtype != 'datetime64[ns]':
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

data['Check-in_dt'] = data.apply(
    lambda row: extract_kurung_datetime(row['Check-in'], row['Date']), axis=1)
data['Check-out_dt'] = data.apply(
    lambda row: extract_kurung_datetime(row['Check-out'], row['Date']), axis=1)

data['work_minutes'] = (
    (data['Check-out_dt'] - data['Check-in_dt']).dt.total_seconds() / 60
)
data['work_minutes'] = data['work_minutes'].apply(lambda x: x if pd.notnull(x) and x > 0 else 0)

# --- Rekap total waktu kerja per Worker, Date, Title
rekap_title = (
    data.groupby(['Worker','Date','Title'], as_index=False)
    .agg(
        total_minutes=('work_minutes', 'sum'),
        total_visit=('work_minutes', 'count'),
        avg_minutes=('work_minutes', 'mean')
    )
)

rekap_title['jam_menit'] = rekap_title['total_minutes'].apply(format_jam_menit)
rekap_title['rata2_per_kunjungan'] = rekap_title['avg_minutes'].apply(format_menit)
print("\n=== Rekap Title/Nama/Jam/Total & Rata-rata Per Kunjungan ===")
print(rekap_title[['Worker', 'Date', 'Title', 'jam_menit', 'total_minutes', 'total_visit', 'rata2_per_kunjungan']].head(20))



=== Rekap Title/Nama/Jam/Total & Rata-rata Per Kunjungan ===
                     Worker       Date                Title jam_menit  \
0       AMIRA MELIANA PUTRI 2025-12-22           Koordinasi  0:18 jam   
1       AMIRA MELIANA PUTRI 2025-12-22  Survey Rumah Tangga  3:17 jam   
2          Ainun Musyarofah 2025-12-22           Koordinasi  0:38 jam   
3          Ainun Musyarofah 2025-12-22  Survey Rumah Tangga  3:04 jam   
4       Al Asri Dwi Salasih 2025-12-22           Koordinasi  0:35 jam   
5       Al Asri Dwi Salasih 2025-12-22  Survey Rumah Tangga  5:13 jam   
6          Alfi Ayuningtias 2025-12-22           Koordinasi  2:44 jam   
7          Alfi Ayuningtias 2025-12-22  Survey Rumah Tangga  2:35 jam   
8      Alya Rizqi Salsabila 2025-12-22           Koordinasi  0:00 jam   
9      Alya Rizqi Salsabila 2025-12-22  Survey Rumah Tangga  0:00 jam   
10  Amalia Cahya Anggraheni 2025-12-22           Koordinasi  1:05 jam   
11  Amalia Cahya Anggraheni 2025-12-22  Survey Rumah Tangga  2

In [ ]:
import re
import datetime

def parse_datetime_to_parentheses(s, year='2025', default_day=None, default_month=None):
    # Kembalikan NaT jika kosong
    if pd.isna(s) or s == '':
        return pd.NaT

    # Kalau berupa datetime.time atau datetime.datetime, jadikan HH:MM
    if isinstance(s, (datetime.time, datetime.datetime)):
        s = s.strftime('%H:%M')

    # Normalisasi string dan double space.
    s_str = str(s).strip().replace('  ', ' ')

    # 1) Pola sudah dalam kurung '(dd Mon HH:MM)' jadi neh yang oe ambil itu yang di dalam kurung. kadang ada dua bagian jam nya (diluar kurung dan dalam kurung atau salah satu)
    m_paren = re.findall(r'\((\d{2}) (\w{3}) (\d{2}):(\d{2})\)', s_str)
    if m_paren:
        day, mon, hh, mm = m_paren[-1]
    else:
        # 2) Pola jam sederhana 'HH:MM' atau 'HH:MM:SS'
        #    Detik opsional akan diabaikan pada output agar jadi HH:MM saja
        m_time = re.match(r'^(\d{1,2}):(\d{2})(?::(\d{2}))?$', s_str)
        if m_time:
            hh, mm = m_time.group(1), m_time.group(2)
            # Perlu default day & month untuk melengkapi tanggal
            if default_day and default_month:
                day = str(default_day).zfill(2)
                mon = str(default_month)
            else:
                return pd.NaT
        else:
            # 3) Dukungan pola lain seperti 'HHMM dd Mon HHMM' (jika ada di data mentah)
            #    Ambil pasangan terakhir 'dd Mon HHMM' menjadi HH:MM
            m_comp = re.findall(r'(\d{2}) (\w{3}) (\d{2})(\d{2})', s_str)
            if m_comp:
                day, mon, hh, mm = m_comp[-1]
            else:
                return pd.NaT

    # Konversi ke datetime lalu format ulang menjadi '(dd Mon HH:MM)'
    try:
        dt = pd.to_datetime(f'{year}-{mon}-{day} {hh}:{mm}', format='%Y-%b-%d %H:%M')
        return f'({dt.strftime("%d %b %H:%M")})'
    except ValueError:
        return pd.NaT

# Pastikan kolom 'Date' ke datetime
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Default day & month dari kolom Date
data['default_day'] = data['Date'].dt.day.astype(str).str.zfill(2)
data['default_month'] = data['Date'].dt.strftime('%b')

# Terapkan ke Check-in & Check-out
data['ci_dt'] = data.apply(
    lambda row: parse_datetime_to_parentheses(
        row['Check-in'], year='2025',
        default_day=row['default_day'],
        default_month=row['default_month']
    ),
    axis=1
)

data['co_dt'] = data.apply(
    lambda row: parse_datetime_to_parentheses(
        row['Check-out'], year='2025',
        default_day=row['default_day'],
        default_month=row['default_month']
    ),
    axis=1
)

# Contoh output
print(data[['Check-in', 'Check-out', 'ci_dt', 'co_dt']].head(10))


               Check-in             Check-out           ci_dt           co_dt
0                   NaN                   NaN             NaT             NaT
1        (22 Dec 12:54)        (22 Dec 13:46)  (22 Dec 12:54)  (22 Dec 13:46)
2  10:32 (22 Dec 09:35)  10:33 (22 Dec 12:00)  (22 Dec 09:35)  (22 Dec 12:00)
3        (22 Dec 09:10)        (22 Dec 09:28)  (22 Dec 09:10)  (22 Dec 09:28)
4        (22 Dec 09:56)        (22 Dec 11:32)  (22 Dec 09:56)  (22 Dec 11:32)
5        (22 Dec 11:40)        (22 Dec 11:53)  (22 Dec 11:40)  (22 Dec 11:53)
6        (22 Dec 12:01)        (22 Dec 13:16)  (22 Dec 12:01)  (22 Dec 13:16)
7        (22 Dec 09:11)        (22 Dec 09:49)  (22 Dec 09:11)  (22 Dec 09:49)
8        (22 Dec 09:51)        (22 Dec 10:43)  (22 Dec 09:51)  (22 Dec 10:43)
9                 10:06                 14:27  (22 Dec 10:06)  (22 Dec 14:27)


In [ ]:
def extract_datetime(s):
    # Ambil jam-tanggal dalam kurung, contoh: "10:26 (03 Sep 10:27)" → "03 Sep 10:27"
    if pd.isnull(s): return pd.NaT
    match = re.search(r'\((\d{2}) (\w{3}) (\d{2}):(\d{2})\)', str(s))
    if not match: return pd.NaT
    day, mon, hour, minute = match.group(1), match.group(2), match.group(3), match.group(4)
    # Format target: "2025-Sep-03 10:27"
    return pd.to_datetime(f"2025-{mon}-{day} {hour}:{minute}", format="%Y-%b-%d %H:%M", errors='coerce')

# Parse isi Check-in & Check-out hanya dari bagian kurung
data['ci_dt'] = data['Check-in'].apply(extract_datetime)
data['co_dt'] = data['Check-out'].apply(extract_datetime)

# Hitung durasi kerja tiap baris (menit)
data['work_minutes'] = (data['co_dt'] - data['ci_dt']).dt.total_seconds() / 60
data['work_minutes'] = data['work_minutes'].fillna(0)

# Rekap total jam kerja per Worker per Date
rekap = (
    data.groupby(['Worker', 'Date'], as_index=False)['work_minutes']
    .sum()
    .assign(total_working_hours=lambda df: (df['work_minutes']/60).round(2))
    .drop(columns='work_minutes')
)

rekap.head()


,Worker,Date,total_working_hours
0,AMIRA MELIANA PUTRI,2025-12-22,3.58
1,Ainun Musyarofah,2025-12-22,3.70
2,Al Asri Dwi Salasih,2025-12-22,1.45
3,Alfi Ayuningtias,2025-12-22,5.32
4,Alya Rizqi Salsabila,2025-12-22,0.00


In [ ]:
def is_time_format(val):
    # Format jam:menit ekstrak, contoh: 10:11. nih kadang ada yang ga ada di kurung.
    return isinstance(val, str) and bool(re.match(r'^\d{2}:\d{2}$', val.strip()))

filtered = data[
    data['Check-in'].apply(is_time_format) & data['Check-out'].apply(is_time_format)
][['Worker','Date', 'Check-in', 'Check-out']]
filtered.head()


,Worker,Date,Check-in,Check-out
9,Al Asri Dwi Salasih,2025-12-22,10:06,14:27
21,Amalia Cahya Anggraheni,2025-12-22,15:13,15:41
138,Yutika Praningsih,2025-12-22,09:51,09:52


In [ ]:
data['Check-in'].head(10)

,Check-in
0,NaN
1,(22 Dec 12:54)
2,10:32 (22 Dec 09:35)
3,(22 Dec 09:10)
4,(22 Dec 09:56)
5,(22 Dec 11:40)
6,(22 Dec 12:01)
7,(22 Dec 09:11)
8,(22 Dec 09:51)
9,10:06


In [ ]:
# Fungsi untuk memeriksa apakah format waktu sesuai (jam:menit)?
def is_time_format(val):
    return isinstance(val, str) and bool(re.match(r'^\d{2}:\d{2}$', val.strip()))
def combine_date_time(date, time):
    return f"({date} {time})"

# Mengonversi kolom Date menjadi datetime dengan format yang benar
data['Date'] = pd.to_datetime(data['Date'], format='%d %b %Y').dt.strftime('%d %b %Y')

data['Check-in'] = data.apply(lambda row: combine_date_time(row['Date'], row['Check-in'])
                              if is_time_format(row['Check-in']) else row['Check-in'], axis=1)
data['Check-out'] = data.apply(lambda row: combine_date_time(row['Date'], row['Check-out'])
                               if is_time_format(row['Check-out']) else row['Check-out'], axis=1)
print(data[['Date', 'Check-in', 'Check-out']].head(10))

          Date              Check-in             Check-out
0  22 Dec 2025                   NaN                   NaN
1  22 Dec 2025        (22 Dec 12:54)        (22 Dec 13:46)
2  22 Dec 2025  10:32 (22 Dec 09:35)  10:33 (22 Dec 12:00)
3  22 Dec 2025        (22 Dec 09:10)        (22 Dec 09:28)
4  22 Dec 2025        (22 Dec 09:56)        (22 Dec 11:32)
5  22 Dec 2025        (22 Dec 11:40)        (22 Dec 11:53)
6  22 Dec 2025        (22 Dec 12:01)        (22 Dec 13:16)
7  22 Dec 2025        (22 Dec 09:11)        (22 Dec 09:49)
8  22 Dec 2025        (22 Dec 09:51)        (22 Dec 10:43)
9  22 Dec 2025   (22 Dec 2025 10:06)   (22 Dec 2025 14:27)


In [ ]:
def is_time_format(val):
    # Memeriksa apakah formatnya adalah jam:menit (misalnya, 13:05)??
    return isinstance(val, str) and bool(re.match(r'^\d{2}:\d{2}$', val.strip()))

# Menggunakan variable ini untuk menyaring data
filtered = data[data['Check-in'].apply(is_time_format) & data['Check-out'].apply(is_time_format)][['Check-in', 'Check-out']]
filtered.head()


,Check-in,Check-out


> ### Next Step!

In [ ]:
# Filter hanya baris dengan check-in atau check-out valid
valid_jobs = data[(data['ci_dt'].notna()) | (data['co_dt'].notna())]

# Rekap total, check-in, dan check-out jobs
rekap_total = valid_jobs.groupby(['Worker','Date'], as_index=False).size().rename(columns={'size':'total_jobs'})

rekap_checkin = valid_jobs[valid_jobs['ci_dt'].notna()].groupby(['Worker','Date']).size().reset_index(name='checkin_jobs')
rekap_checkout = valid_jobs[valid_jobs['co_dt'].notna()].groupby(['Worker','Date']).size().reset_index(name='checkout_jobs')

step3_result = rekap_total.merge(rekap_checkin,on=['Worker','Date'],how='left').merge(rekap_checkout,on=['Worker','Date'],how='left')
step3_result[['checkin_jobs', 'checkout_jobs']] = step3_result[['checkin_jobs', 'checkout_jobs']].fillna(0)

step3_result['persen_checkin'] = (step3_result['checkin_jobs']/step3_result['total_jobs']*100).round(2)
step3_result['persen_checkout'] = (step3_result['checkout_jobs']/step3_result['total_jobs']*100).round(2)

step3_result['Status'] = step3_result.apply(lambda row: 'Complete' if row['persen_checkin'] == 100 and row['persen_checkout'] == 100 else 'Incomplete', axis=1)
step3_result[['Worker','Date','total_jobs','checkin_jobs','persen_checkin','checkout_jobs','persen_checkout','Status']].head()


,Worker,Date,total_jobs,checkin_jobs,persen_checkin,checkout_jobs,persen_checkout,Status
0,AMIRA MELIANA PUTRI,22 Dec 2025,3,3,100.0,3,100.0,Complete
1,Ainun Musyarofah,22 Dec 2025,4,4,100.0,4,100.0,Complete
2,Al Asri Dwi Salasih,22 Dec 2025,2,2,100.0,2,100.0,Complete
3,Alfi Ayuningtias,22 Dec 2025,4,4,100.0,4,100.0,Complete
4,Amalia Cahya Anggraheni,22 Dec 2025,3,3,100.0,3,100.0,Complete


In [ ]:
rekap_total = data.groupby(['Worker', 'Date'], as_index=False).size().rename(columns={'size': 'total_jobs'})

# Hitung checkin_jobs: jumlah baris dengan ci_dt notna
rekap_checkin = data[data['ci_dt'].notna()].groupby(['Worker', 'Date']).size().reset_index(name='checkin_jobs')

# Hitung checkout_jobs: jumlah baris dengan co_dt notna
rekap_checkout = data[data['co_dt'].notna()].groupby(['Worker', 'Date']).size().reset_index(name='checkout_jobs')

# Hitung NaN Check-in: jumlah baris dengan ci_dt isna
nan_checkin = data[data['ci_dt'].isna()].groupby(['Worker', 'Date']).size().reset_index(name='NaN Check-in')

# Hitung NaN Check-out: jumlah baris dengan co_dt isna
nan_checkout = data[data['co_dt'].isna()].groupby(['Worker', 'Date']).size().reset_index(name='NaN Check-out')

# Merge hasil rekapitulasi
step3_result = rekap_total.merge(rekap_checkin, on=['Worker', 'Date'], how='left') \
                          .merge(rekap_checkout, on=['Worker', 'Date'], how='left') \
                          .merge(nan_checkin, on=['Worker', 'Date'], how='left') \
                          .merge(nan_checkout, on=['Worker', 'Date'], how='left')

# Fill NaN values with 0 for check-in, check-out, dan kolom NaN baru
step3_result[['checkin_jobs', 'checkout_jobs', 'NaN Check-in', 'NaN Check-out']] = \
    step3_result[['checkin_jobs', 'checkout_jobs', 'NaN Check-in', 'NaN Check-out']].fillna(0)

# Calculate percentage for check-in and check-out
step3_result['persen_checkin'] = (step3_result['checkin_jobs'] / step3_result['total_jobs'] * 100).round(2)
step3_result['persen_checkout'] = (step3_result['checkout_jobs'] / step3_result['total_jobs'] * 100).round(2)

# Add Status column based on completeness check
step3_result['Status'] = step3_result.apply(lambda row: 'Complete' if row['persen_checkin'] == 100 and row['persen_checkout'] == 100 else 'Incomplete', axis=1)

# Display the final result (dengan kolom baru)
step3_result[['Worker', 'Date', 'total_jobs', 'checkin_jobs', 'NaN Check-in', 'persen_checkin',
                    'checkout_jobs', 'NaN Check-out', 'persen_checkout', 'Status']].head()

,Worker,Date,total_jobs,checkin_jobs,NaN Check-in,persen_checkin,checkout_jobs,NaN Check-out,persen_checkout,Status
0,AMIRA MELIANA PUTRI,22 Dec 2025,4,3.0,1.0,75.0,3.0,1.0,75.0,Incomplete
1,Ainun Musyarofah,22 Dec 2025,4,4.0,0.0,100.0,4.0,0.0,100.0,Complete
2,Al Asri Dwi Salasih,22 Dec 2025,4,2.0,2.0,50.0,2.0,2.0,50.0,Incomplete
3,Alfi Ayuningtias,22 Dec 2025,4,4.0,0.0,100.0,4.0,0.0,100.0,Complete
4,Alya Rizqi Salsabila,22 Dec 2025,4,0.0,4.0,0.0,0.0,4.0,0.0,Incomplete


> ### Last Check In / Out

In [ ]:
# Ambil jam pertama check-in (awal kerja) per Worker per hari
first_checkin = (
    data[data['ci_dt'].notna()]
    .groupby(['Worker','Date'], as_index=False)['ci_dt']
    .min()
    .rename(columns={'ci_dt':'first_checkin'})
)

# Ambil jam terakhir check-out (akhir kerja) per Worker per hari
last_checkout = (
    data[data['co_dt'].notna()]
    .groupby(['Worker','Date'], as_index=False)['co_dt']
    .max()
    .rename(columns={'co_dt':'last_checkout'})
)

# Gabungkan kedua informasi
jam_mulai_selesai = pd.merge(first_checkin, last_checkout, on=['Worker','Date'], how='outer')

# Tampilkan hanya jamnya dalam format HH.MM (gunakan .dt.strftime)
jam_mulai_selesai['first_ci_time'] = jam_mulai_selesai['first_checkin'].dt.strftime('%H.%M')
jam_mulai_selesai['last_co_time']  = jam_mulai_selesai['last_checkout'].dt.strftime('%H.%M')

# Output final sesuai contoh (format jam):
jam_mulai_selesai[['Worker','Date','first_ci_time','last_co_time']].head()

,Worker,Date,first_ci_time,last_co_time
0,AMIRA MELIANA PUTRI,22 Dec 2025,09.10,13.46
1,Ainun Musyarofah,22 Dec 2025,09.11,13.16
2,Al Asri Dwi Salasih,22 Dec 2025,09.09,10.43
3,Alfi Ayuningtias,22 Dec 2025,09.04,12.02
4,Amalia Cahya Anggraheni,22 Dec 2025,08.50,14.02


> ### Rentang waktu antar Job!

In [ ]:
import pandas as pd
import re

# Salin data
df = data.copy()

# --- Parsing waktu aktual dari format "(27 Sep HH:MM)" ---
def parse_actual_time(cell):
    if pd.isna(cell):
        return pd.NaT
    match = re.search(r'\((\d{1,2})\s+([A-Za-z]{3})\s+(\d{1,2}):(\d{2})\)', str(cell))
    if match:
        day, month, hour, minute = match.groups()
        time_str = f"2025 {day} {month} {hour}:{minute}"
        return pd.to_datetime(time_str, format='%Y %d %b %H:%M', errors='coerce')
    else:
        return pd.NaT

# Ekstrak waktu sebenarnya
df['CheckInActual'] = df['Check-in'].apply(parse_actual_time)
df['CheckOutActual'] = df['Check-out'].apply(parse_actual_time)

# Filter dan urutkan
df_valid = df.dropna(subset=['CheckInActual', 'CheckOutActual']).copy()
df_valid = df_valid.sort_values(['Worker', 'CheckInActual']).reset_index(drop=True)

# Ambil data job berikutnya
df_valid['NextCheckIn'] = df_valid.groupby('Worker')['CheckInActual'].shift(-1)
df_valid['NextTitle'] = df_valid.groupby('Worker')['Title'].shift(-1)

# Hanya transisi yang valid
trans = df_valid.dropna(subset=['NextCheckIn']).copy()
trans['Break_min'] = (trans['NextCheckIn'] - trans['CheckOutActual']).dt.total_seconds() / 60
trans['TransNum'] = trans.groupby('Worker').cumcount() + 1

# --- Kumpulkan per worker ---
records = []
for worker, group in trans.groupby('Worker'):
    # Ambil Title dari job PERTAMA (yang menghasilkan CO1)
    title1 = group.iloc[0]['Title']

    rec = {'Worker': worker, 'Title1': title1}
    total_break = 0.0
    for _, row in group.iterrows():
        n = int(row['TransNum'])
        break_val = row['Break_min']
        total_break += break_val

        rec[f'CO{n}'] = row['CheckOutActual']
        rec[f'CI{n+1}'] = row['NextCheckIn']
        rec[f'Title{n+1}'] = row['NextTitle']
        rec[f'Break{n}_min'] = break_val

    rec['Total_Break_Min'] = total_break
    records.append(rec)

# Buat DataFrame
result = pd.DataFrame(records)

# --- Hitung Average_Break_Min per baris ---
break_cols = [col for col in result.columns if col.startswith('Break') and col.endswith('_min')]
if break_cols:
    result['Average_Break_Min'] = result[break_cols].mean(axis=1)
else:
    result['Average_Break_Min'] = 0.0  # atau pd.NA jika lebih sesuai

# --- Urutkan kolom dengan Title1 di awal dan Average_Break_Min di akhir ---
if not result.empty:
    # Cari jumlah maksimum transisi (berdasarkan kolom CO)
    co_cols = [col for col in result.columns if col.startswith('CO')]
    if co_cols:
        max_n = max(int(col[2:]) for col in co_cols)
    else:
        max_n = 0

    ordered = ['Worker', 'Title1']
    for n in range(1, max_n + 1):
        ordered.extend([f'CO{n}', f'CI{n+1}', f'Title{n+1}', f'Break{n}_min'])
    ordered.extend(['Total_Break_Min', 'Average_Break_Min'])

    # Pastikan hanya kolom yang benar-benar ada dalam DataFrame
    ordered = [col for col in ordered if col in result.columns]
    result = result[ordered]

# Tampilkan hasil
result.head()

,Worker,Title1,CO1,CI2,Title2,Break1_min,CO2,CI3,Title3,Break2_min,CO3,CI4,Title4,Break3_min,Total_Break_Min,Average_Break_Min
0,AMIRA MELIANA PUTRI,Koordinasi,2025-12-22 09:28:00,2025-12-22 09:35:00,Survey Rumah Tangga,7.0,2025-12-22 12:00:00,2025-12-22 12:54:00,Survey Rumah Tangga,54.0,NaT,NaT,NaN,NaN,61.0,30.500000
1,Ainun Musyarofah,Koordinasi,2025-12-22 09:49:00,2025-12-22 09:56:00,Survey Rumah Tangga,7.0,2025-12-22 11:32:00,2025-12-22 11:40:00,Survey Rumah Tangga,8.0,2025-12-22 11:53:00,2025-12-22 12:01:00,Survey Rumah Tangga,8.0,23.0,7.666667
2,Al Asri Dwi Salasih,Koordinasi,2025-12-22 09:44:00,2025-12-22 09:51:00,Survey Rumah Tangga,7.0,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN,7.0,7.000000
3,Alfi Ayuningtias,Koordinasi,2025-12-22 11:48:00,2025-12-22 09:22:00,Survey Rumah Tangga,-146.0,2025-12-22 10:04:00,2025-12-22 10:05:00,Survey Rumah Tangga,1.0,2025-12-22 10:58:00,2025-12-22 11:02:00,Survey Rumah Tangga,4.0,-141.0,-47.000000
4,Amalia Cahya Anggraheni,Koordinasi,2025-12-22 09:55:00,2025-12-22 10:32:00,Survey Rumah Tangga,37.0,2025-12-22 11:33:00,2025-12-22 12:40:00,Survey Rumah Tangga,67.0,NaT,NaT,NaN,NaN,104.0,52.000000


In [ ]:
rentang_job = result

In [ ]:
import re

def extract_kurung_datetime(s, tanggal=None, tahun_default='2025'):
    if pd.isnull(s): return pd.NaT
    s = str(s).strip()
    # Kurung
    match_kurung = re.match(r'\((\d{2}) (\w{3}) (\d{2}):(\d{2})\)', s)
    if match_kurung:
        day, month, hour, minute = match_kurung.groups()
        tahun = tahun_default
        return pd.to_datetime(f"{tahun}-{month}-{day} {hour}:{minute}", format="%Y-%b-%d %H:%M", errors='coerce')
    # Kombinasi
    match_comb = re.search(r'\((\d{2}) (\w{3}) (\d{2}):(\d{2})\)', s)
    if match_comb:
        day, month, hour, minute = match_comb.groups()
        tahun = tahun_default
        return pd.to_datetime(f"{tahun}-{month}-{day} {hour}:{minute}", format="%Y-%b-%d %H:%M", errors='coerce')
    # Jam tunggal
    match_jam = re.match(r'^(\d{2}):(\d{2})$', s)
    if match_jam and tanggal is not None:
        if isinstance(tanggal, pd.Timestamp):
            day = tanggal.day
            month = tanggal.strftime('%b')
            tahun = tanggal.year
        else:
            parts = str(tanggal).split()
            if len(parts) == 3:
                day, month, tahun = parts
        hour, minute = match_jam.groups()
        return pd.to_datetime(f"{tahun}-{month}-{day} {hour}:{minute}", format="%Y-%b-%d %H:%M", errors='coerce')
    return pd.NaT

def format_menit(menit_float):
    if pd.isnull(menit_float) or menit_float <= 0:
        return "0:00 menit"
    total_seconds = int(round(menit_float * 60))
    menit = total_seconds // 60
    detik = total_seconds % 60
    return f"{menit}:{detik:02d} menit"

# Pastikan 'Date' dalam datetime
if data['Date'].dtype != 'datetime64[ns]':
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# --- Parsing waktu ---
data['Check-in_dt'] = data.apply(
    lambda row: extract_kurung_datetime(row['Check-in'], row['Date']), axis=1)
data['Check-out_dt'] = data.apply(
    lambda row: extract_kurung_datetime(row['Check-out'], row['Date']), axis=1)

# --- Hitung durasi, pastikan positive ---
data['work_minutes_raw'] = (
    (data['Check-out_dt'] - data['Check-in_dt']).dt.total_seconds() / 60
)
data['work_minutes'] = data['work_minutes_raw'].apply(lambda x: x if pd.notnull(x) and x >= 0 else 0)

# --- Rekap total jam kerja per Worker per hari (format jam:menit) ---
rekap_worktime = (
    data.groupby(['Worker','Date'], as_index=False)['work_minutes']
    .sum()
)
rekap_worktime['jam_menit'] = rekap_worktime['work_minutes'].apply(
    lambda x: f"{int(x//60)}:{int(x%60):02d} jam"
)

# --- Hitung rata-rata hanya dari baris positif ---
avg_per_title = (
    data[data['work_minutes'] > 0].groupby(['Worker','Date','Title'], as_index=False)['work_minutes']
    .mean()
)
avg_per_title = avg_per_title.rename(columns={'work_minutes': 'rata2_title_menit'})
avg_per_title['rata2_title_menit'] = avg_per_title['rata2_title_menit'].apply(format_menit)

rekap_worktime['total_minutes'] = rekap_worktime['work_minutes'].astype(int)
df_final = avg_per_title.merge(
    rekap_worktime[['Worker','Date','jam_menit','total_minutes']],
    on=['Worker','Date'],
    how='left'
)
# Total jam:menit dan menit KERJA per (Worker, Date, Title)
total_minutes_per_title = (
    data.groupby(['Worker','Date','Title'], as_index=False)['work_minutes']
    .sum()
    .rename(columns={'work_minutes': 'total_minutes'})
)
total_minutes_per_title['jam_menit'] = total_minutes_per_title['total_minutes'].apply(
    lambda x: f"{int(x//60)}:{int(x%60):02d} jam"
)

# Gabungkan dengan rata-rata menit kunjungan (rata2_title_menit)
df_final = avg_per_title.merge(
    total_minutes_per_title,
    on=['Worker','Date','Title'],
    how='left'
)

# --- Tampilkan cek summary ---
print("\n=== Rekap Waktu Kerja Title per Hari (Valid Durasi) ===")
df_final[['Worker','Date','Title','jam_menit','total_minutes','rata2_title_menit']].head(2)


=== Rekap Waktu Kerja Title per Hari (Valid Durasi) ===


,Worker,Date,Title,jam_menit,total_minutes,rata2_title_menit
0,AMIRA MELIANA PUTRI,2025-12-22,Koordinasi,0:18 jam,18.0,18:00 menit
1,AMIRA MELIANA PUTRI,2025-12-22,Survey Rumah Tangga,3:17 jam,197.0,98:30 menit


In [ ]:
# Rekap total work_minutes per Worker per Date
rekap_worktime = (
    data.groupby(['Worker','Date'], as_index=False)['work_minutes']
    .sum()
)

# Buat satu kolom baru dengan format "2:18 jam"
rekap_worktime['jam_menit'] = rekap_worktime['work_minutes'].apply(
    lambda x: f"{int(x // 60)}:{int(x % 60):02d} jam"
)
# Tampilkan
rekap_worktime[['Worker', 'Date', 'jam_menit']].head()

,Worker,Date,jam_menit
0,AMIRA MELIANA PUTRI,2025-12-22,3:35 jam
1,Ainun Musyarofah,2025-12-22,3:42 jam
2,Al Asri Dwi Salasih,2025-12-22,1:27 jam
3,Alfi Ayuningtias,2025-12-22,5:19 jam
4,Alya Rizqi Salsabila,2025-12-22,0:00 jam


In [ ]:
print("df_final['Date'].dtype:", df_final['Date'].dtype)
print("step3_result['Date'].dtype:", step3_result['Date'].dtype)

df_final['Date'].dtype: datetime64[ns]
step3_result['Date'].dtype: object


In [ ]:
# Total jobs per Title
rekap_total_per_title = valid_jobs.groupby(['Worker','Date','Title'], as_index=False).size().rename(columns={'size':'total_jobs'})

# Check-in jobs per Title
rekap_checkin_per_title = valid_jobs[valid_jobs['ci_dt'].notna()].groupby(['Worker','Date','Title']).size().reset_index(name='checkin_jobs')

# Check-out jobs per Title
rekap_checkout_per_title = valid_jobs[valid_jobs['co_dt'].notna()].groupby(['Worker','Date','Title']).size().reset_index(name='checkout_jobs')

# Gabungkan
step3_result_per_title = rekap_total_per_title.merge(
    rekap_checkin_per_title, on=['Worker','Date','Title'], how='left'
).merge(
    rekap_checkout_per_title, on=['Worker','Date','Title'], how='left'
)

# Isi NaN dengan 0
step3_result_per_title[['checkin_jobs', 'checkout_jobs']] = step3_result_per_title[['checkin_jobs', 'checkout_jobs']].fillna(0)

# Hitung persentase
step3_result_per_title['persen_checkin'] = (step3_result_per_title['checkin_jobs'] / step3_result_per_title['total_jobs'] * 100).round(2)
step3_result_per_title['persen_checkout'] = (step3_result_per_title['checkout_jobs'] / step3_result_per_title['total_jobs'] * 100).round(2)

# Hapus kolom Status (karena kamu minta hapus)
# step3_result_per_title = step3_result_per_title.drop(columns=['Status'])  # tidak ada dulu, jadi skip

In [ ]:
# Pastikan kolom Date dalam format datetime (opsional, karena nanti di-drop)
df_final['Date'] = pd.to_datetime(df_final['Date'], errors='coerce')
step3_result_per_title['Date'] = pd.to_datetime(step3_result_per_title['Date'], errors='coerce')

# Gabungkan data seperti biasa (masih per tanggal dulu, lalu kita agregasi)
gabungan_per_tanggal = df_final.merge(
    step3_result_per_title[['Worker', 'Date', 'Title', 'total_jobs', 'checkin_jobs', 'persen_checkin', 'checkout_jobs', 'persen_checkout']],
    on=['Worker', 'Date', 'Title'],
    how='left'
)

# Isi NaN dengan 0 untuk kolom numerik
kolom_numerik = ['total_jobs', 'checkin_jobs', 'checkout_jobs', 'persen_checkin', 'persen_checkout']
gabungan_per_tanggal[kolom_numerik] = gabungan_per_tanggal[kolom_numerik].fillna(0)

# Sekarang, AGREGASI SEMUA DATA PER Worker + Title (abaikan Date)
gabungan_durasi_cico = gabungan_per_tanggal.groupby(['Worker', 'Title'], as_index=False).agg(
    total_minutes=('total_minutes', 'sum'),
    total_jobs=('total_jobs', 'sum'),
    checkin_jobs=('checkin_jobs', 'sum'),
    checkout_jobs=('checkout_jobs', 'sum'),
    # Catatan: persentase tidak bisa dijumlah, jadi kita hitung ulang nanti
)

# Hitung ulang persentase check-in dan check-out berdasarkan total
gabungan_durasi_cico['persen_checkin'] = (
    gabungan_durasi_cico['checkin_jobs'] / gabungan_durasi_cico['total_jobs'].replace(0, 1) * 100
).round(2)
gabungan_durasi_cico['persen_checkout'] = (
    gabungan_durasi_cico['checkout_jobs'] / gabungan_durasi_cico['total_jobs'].replace(0, 1) * 100
).round(2)

# Isi 0 jika total_jobs = 0
mask_no_jobs = gabungan_durasi_cico['total_jobs'] == 0
gabungan_durasi_cico.loc[mask_no_jobs, ['persen_checkin', 'persen_checkout']] = 0

# Hitung rata-rata menit per job
gabungan_durasi_cico['rata2_title_menit'] = (
    gabungan_durasi_cico['total_minutes'] / gabungan_durasi_cico['total_jobs'].replace(0, 1)
)
gabungan_durasi_cico['rata2_title_menit'] = gabungan_durasi_cico['rata2_title_menit'].where(
    gabungan_durasi_cico['total_jobs'] > 0, 0
).round(2)

# Tambahkan kolom display "menit:detik"
def menit_ke_format(menit):
    if pd.isna(menit) or menit <= 0:
        return "0:00"
    total_detik = int(round(menit * 60))
    m, s = divmod(total_detik, 60)
    return f"{m}:{s:02d}"

gabungan_durasi_cico['durasi_format'] = gabungan_durasi_cico['rata2_title_menit'].apply(menit_ke_format)

# Urutkan kolom
gabungan_durasi_cico = gabungan_durasi_cico[[
    'Worker', 'Title',
    'total_minutes', 'rata2_title_menit', 'durasi_format',
    'total_jobs', 'checkin_jobs', 'persen_checkin',
    'checkout_jobs', 'persen_checkout'
]]

print("\n=== Data Agregat per Worker & Title (SEMUA TANGGAL DIGABUNG) ===")
print(gabungan_durasi_cico.head(20))


=== Data Agregat per Worker & Title (SEMUA TANGGAL DIGABUNG) ===
                     Worker                Title  total_minutes  \
0       AMIRA MELIANA PUTRI           Koordinasi           18.0   
1       AMIRA MELIANA PUTRI  Survey Rumah Tangga          197.0   
2          Ainun Musyarofah           Koordinasi           38.0   
3          Ainun Musyarofah  Survey Rumah Tangga          184.0   
4       Al Asri Dwi Salasih           Koordinasi           35.0   
5       Al Asri Dwi Salasih  Survey Rumah Tangga           52.0   
6          Alfi Ayuningtias           Koordinasi          164.0   
7          Alfi Ayuningtias  Survey Rumah Tangga          155.0   
8   Amalia Cahya Anggraheni           Koordinasi           65.0   
9   Amalia Cahya Anggraheni  Survey Rumah Tangga          143.0   
10        Anggita Eka Putri           Koordinasi          106.0   
11        Anggita Eka Putri  Survey Rumah Tangga          117.0   
12     Ashilla Nadiya Amany           Koordinasi          345.0

> ### Scoring

In [ ]:
# # Filter hanya data dengan Title 'Kunjungan Rumah'
# gabungan_durasi_cico = gabungan_durasi_cico[gabungan_durasi_cico['Title'] == 'Kunjungan Rumah'].copy()

# # Nilai acuan tetap dari rumus Excel
# AVERAGE_FIXED = 3.788

# # --- 1️⃣ Pastikan durasi dalam menit numerik dulu (kalau belum)
# gabungan_durasi_cico['rata2_title_menit'] = pd.to_numeric(
#     gabungan_durasi_cico['rata2_title_menit'], errors='coerce'
# )

# # --- 2️⃣ Hitung scoring pakai angka menit sebelum diformat (tanpa pembulatan logika)
# def hitung_scoring(row):
#     val = row['rata2_title_menit']
#     if pd.isna(val):
#         return 0
#     if val < AVERAGE_FIXED:
#         score = (val / AVERAGE_FIXED) * 100
#     else:
#         score = 200 - (val / AVERAGE_FIXED) * 100
#     return max(0, min(100, score))

# gabungan_durasi_cico['scoring'] = gabungan_durasi_cico.apply(hitung_scoring, axis=1)

# # --- 3️⃣ Ranking dari skor tertinggi ke terendah
# gabungan_durasi_cico['ranking'] = gabungan_durasi_cico['scoring'].rank(
#     method='min', ascending=False
# ).astype(int)

# # --- 4️⃣ Format rata2_title_menit jadi "menit:detik"
# def format_menit_detik(val):
#     if pd.isna(val):
#         return "0:00"
#     total_detik = int(round(val * 60))
#     menit = total_detik // 60
#     detik = total_detik % 60
#     return f"{menit}:{detik:02d}"

# gabungan_durasi_cico['rata2_title_menit'] = gabungan_durasi_cico['rata2_title_menit'].apply(format_menit_detik)

# # --- 5️⃣ Format tampilan scoring ke 2 desimal (tanpa ubah nilai ranking)
# gabungan_durasi_cico['scoring'] = gabungan_durasi_cico['scoring'].round(2)

# # --- 6️⃣ Hapus kolom tidak relevan
# gabungan_durasi_cico = gabungan_durasi_cico.drop(columns=['Q1', 'Q3', 'durasi_format'], errors='ignore')

# # --- 7️⃣ Urutkan & reset index
# gabungan_durasi_cico = gabungan_durasi_cico.sort_values(by='ranking', ascending=True).reset_index(drop=True)

# print("✅ Scoring & Ranking selesai — skor tampil 2 desimal, ranking presisi dan format waktu menit:detik!")


> ### TO EXCEL

In [ ]:
import pandas as pd
import numpy as np
from openpyxl.styles import Font

# ===================================================
# 1️⃣ Urutan Worker (Final)
# ===================================================
worker_order = [
    "Ema Pangestika","Alya Rizqi Salsabila","Vianty Ayu Lestary","Novanda Nurliani Putri","Dona Palupi Utari",
    "Anggi Ardiana Saputri","Eti Suharni","Amalia Cahya Anggraheni","Yufiatun Nurkhasanah","Amira Meliana Putri",
    "Tri Baroroh Ratnaningsih","Sri Wigati","Laela Dwi Puspita Sari","Elfa Sari Pambayun","Shinta Maharani Putri",
    "Endah Rahma Setyowati","Ainun Musyarofah","Eva Aprilia Dewi","Tri Setio Utami","Ravinda Tyas Kusumaningrum",
    "Ratna Aji Prastika","Rini Wijayanti","Fina Udhul Masitoh","Anggita Eka Putri","Galuh Lalitadhira Anindita",
    "Intan Supraptiningsih","Rizky Nuramalia","Etri Nur Susanti","Dwi Pangesti","Hani Herlina Agne Saputri",
    "Ummu Mardiyatun","Fitria Prila Rinawati","Lisa Kurniati","Adinda Nur Indriani","Nawang Sasi Chotniati",
    "Yuyun Khikmatin","Elia Vionita","Al Asri Dwi Salasih","Yutika Praningsih","Melina Nur Afifah",
    "Raflina Nurwanda Ningsih","Uti Andriyani","Dea Dwi Oktianti","Awis Damayanti","Safira Almadani",
    "Nabila Eka Rahayu","Ani Kurniawati","Rena Wulandari","Alfi Ayuningtias","Kharisma Salsabila",
    "Dicki Mardiyanti","Ashilla Nadiya Amany","Sofwa Aulia Rahmawati","Sarah Faiqoh Nugroho",
    "Intan Nur Laelasari","Chintya Feronica"
]

# ===================================================
# 2️⃣ Mapping ID Saset
# ===================================================
df_saset = pd.DataFrame({
    "Worker": worker_order,
    "ID Sahabat Sehat": [
        "110820250040","110820250027","110820250030","110820250024","110820250041","110820250050",
        "110820250056","110820250026","110820250005","110820250053","110820250035","110820250011",
        "110820250022","110820250012","110820250038","110820250042","110820250054","110820250031",
        "110820250049","110820250019","110820250046","110820250055","110820250010","110820250036",
        "110820250016","110820250015","110820250014","110820250007","110820250008","110820250058",
        "110820250013","110820250052","110820250025","110820250037","110820250044","110820250017",
        "110820250029","110820250002","110820250032","110820250020","110820250059","110820250004",
        "110820250033","110820250009","110820250047","110820250023","110820250006","110820250045",
        "110820250051","110820250048","110820250021","110820250028","110820250001","110820250039",
        "110820250034","110820250018"
    ]
})

# ===================================================
# 3️⃣ Standardisasi Worker
# ===================================================
def standardize_name(name):
    if pd.isna(name): return ""
    name = ' '.join(str(name).strip().split())
    low = name.lower().replace(" ", "")
    for w in worker_order:
        if w.lower().replace(" ", "") == low:
            return w
    return name.title()

def normalisasi_worker(df):
    df = df.copy()
    df['Worker'] = df['Worker'].apply(standardize_name)
    return df

# ===================================================
# 4️⃣ Hitung Completeness Score
# ===================================================
def hitung_completeness_score(data):
    df = normalisasi_worker(data.copy())
    df['Status'] = df['Status'].astype(str).str.strip().str.title()

    counts = df.groupby(['Worker','Status']).size().unstack(fill_value=0).reset_index()
    for c in ['Success','Scheduled','Issue']:
        if c not in counts.columns: counts[c] = 0

    counts['total_jobs'] = counts['Success'] + counts['Issue'] + counts['Scheduled']
    counts['Final Score'] = counts.apply(
        lambda x: (x['Success'] / (x['total_jobs'] - x['Issue'])) * 100
        if (x['total_jobs'] - x['Issue']) > 0 else 0, axis=1
    ).round(2)

    counts = counts[counts['Final Score'] > 0]   # remove zero scorers
    return counts[['Worker','Success','Scheduled','Issue','total_jobs','Final Score']]

# ===================================================
# 5️⃣ Check in/out Score
# ===================================================
def hitung_check_inout(data):
    df = normalisasi_worker(data.copy())
    df['Status'] = df['Status'].astype(str).str.strip().str.title()
    df = df[df['Status'] == 'Success']

    summary = df.groupby('Worker').agg(
        total_jobs=('Worker','count'),
        checkin_jobs=('Check-in', lambda x: x.notna().sum()),
        checkout_jobs=('Check-out', lambda x: x.notna().sum())
    ).reset_index()

    summary['score'] = (((summary['checkin_jobs']/summary['total_jobs']*100) +
                         (summary['checkout_jobs']/summary['total_jobs']*100)) / 2).round(2)

    summary = summary[summary['score'] > 0]
    return summary[['Worker','total_jobs','checkin_jobs','checkout_jobs','score']]

# ===================================================
# 6️⃣ Hitung DWS
# ===================================================
def hitung_dws(complete, inout):
    merged = pd.merge(complete[['Worker','Final Score']], inout[['Worker','score']], on='Worker', how='inner')
    merged['DWS'] = ((merged['Final Score'] + merged['score']) / 2).round(2)
    merged['Ranking'] = merged['DWS'].rank(method='min', ascending=False).astype(int)
    return merged[['Worker','Final Score','score','DWS','Ranking']]

# ===================================================
# 7️⃣ SIMPAN KE EXCEL + Summarize full
# ===================================================
def simpan_ke_excel(data, output_file="Summary_Job_Report_Completeness.xlsx"):
    complete = hitung_completeness_score(data)
    inout = hitung_check_inout(data)
    dws = hitung_dws(complete, inout)

    saset_filtered = df_saset[df_saset["Worker"].isin(dws["Worker"])]

    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        complete.to_excel(writer, sheet_name='Completeness', index=False)
        inout.to_excel(writer, sheet_name='Check In Out Score', index=False)
        dws.to_excel(writer, sheet_name='DWS', index=False)
        saset_filtered.to_excel(writer, sheet_name='Saset', index=False)

        # 🔥 Summarize + ID Sahabat Sehat
        summarize = dws.merge(df_saset, on="Worker", how="left")[[
            "Worker",
            "ID Sahabat Sehat",
            "Final Score",
            "score",
            "DWS",
            "Ranking"
        ]]
        summarize.to_excel(writer, sheet_name='Summarize', index=False)

    print("🔥 DONE — File Berhasil Dibuat:", output_file)

# ===================================================
# 8️⃣ MAIN RUN
# ===================================================
if __name__ == "__main__":
    input_file = "jobs-report 22 Desember 2025.xlsx"
    output_file = "Summary_Job_Report_Completeness.xlsx"

    print(f"📂 Load data dari: {input_file}")
    data = pd.read_excel(input_file)
    simpan_ke_excel(data, output_file)


📂 Load data dari: jobs-report 22 Desember 2025.xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


🔥 DONE — File Berhasil Dibuat: Summary_Job_Report_Completeness.xlsx


In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Font

def ensure_df(x, title=None):
    if isinstance(x, pd.DataFrame):
        return x
    elif isinstance(x, pd.Series):
        return x.to_frame()
    else:
        return pd.DataFrame({title if title else "Value": [x]})

tables = {
    'Jam Mulai dan Selesai Kerja per Hari': jam_mulai_selesai[['Worker','Date','first_ci_time','last_co_time']],
    'Total Jam Kerja Per Hari': rekap_worktime[['Worker', 'Date', 'jam_menit']],
    'Total Kunjungan Full': df_final[['Worker','Date','Title','jam_menit','total_minutes','rata2_title_menit']],
    'Persentase Penyelesaian Target': step3_result[['Worker', 'Date', 'total_jobs', 'checkin_jobs', 'NaN Check-in', 'persen_checkin',
                    'checkout_jobs', 'NaN Check-out', 'persen_checkout', 'Status']],
    'Final Score':gabungan_durasi_cico,
    'Rentang Job': rentang_job

}


save_path = 'DWS_22_Desember_all_reports.xlsx'

with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
    start_row = 0
    for title, table in tables.items():
        df = ensure_df(table, title).copy()

        if 'NIK' in df.columns:
            df['NIK'] = df['NIK'].astype(str).apply(lambda x: f"\u00A0{x}")
        if not 'Sheet1' in writer.book.sheetnames:
            writer.book.create_sheet('Sheet1')
        worksheet = writer.sheets['Sheet1'] if 'Sheet1' in writer.sheets else writer.book['Sheet1']

        worksheet.cell(row=start_row + 1, column=1, value=title)
        start_row += 1

        df.to_excel(writer, sheet_name='Sheet1', startrow=start_row, index=False)
        start_row += len(df) + 5  # jarak antar tabel
wb = load_workbook(save_path)
ws = wb['Sheet1']
row = 1
for title, table in tables.items():
    ws.cell(row=row, column=1).font = Font(size=18)
    row += 1
    df = ensure_df(table, title)
    row += len(df) + 5
wb.save(save_path)

print(f"File laporan berhasil disimpan di: {save_path}")

File laporan berhasil disimpan di: DWS_22_Desember_all_reports.xlsx
